# Build a Crew to Tailor Job Applications

Built your first multi-agent system.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [20]:
# Warning control
import warnings
# Removed warnings filter
# Configure logging to use default settings
import logging
logging.basicConfig(level=logging.WARNING)

- Import libraries, APIs and LLM

In [28]:
import os
from crewai import Agent, Task, Crew
from langchain_openai import ChatOpenAI
from chromadb import Client
from chromadb.config import Settings
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI


os.environ["OPENAI_API_KEY"] = ""  # Insert your OpenAI API key here/empty if you are using local
os.environ["LLM_BASE_URL"] = "http://localhost:11434"  # Local LLM endpoint
os.environ["OTEL_SDK_DISABLED"] = "true"
# Configure Google Gemini API

os.environ["GOOGLE_API_KEY"] = "KEY"
os.environ["GEMINI_API_KEY"] = "KEY"

llm = ChatOpenAI(
    model="ollama/falcon",
    base_url=os.environ["LLM_BASE_URL"]
)

# # Load the google gemini api key
# google_api_key = os.getenv("GOOGLE_API_KEY")
# genai.configure(api_key=google_api_key)
#
# # Set gemini pro as llm
# llm = ChatGoogleGenerativeAI(
#         model="gemini-pro", verbose=True, temperature=0.5, google_api_key=google_api_key
# )

resume_file_path = 'resume.md'

In [30]:
try:
    with open(resume_file_path, 'r', encoding='utf-8') as f:
        resume_content = f.read()
        print("Resume file loaded successfully!")
except FileNotFoundError:
    raise Exception(f"File not found: {resume_file_path}")

Resume file loaded successfully!


## crewAI Tools

In [31]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path=resume_file_path)
semantic_search_resume = MDXSearchTool( mdx=resume_file_path,
    config=dict(
        llm=dict(
            provider="ollama", # Options include google, openai, anthropic, llama2, etc.
            config=dict(
                model="falcon",
                # Optional parameters can be included here.
                # temperature=0.5,
                # top_p=1,
                # stream=true,
            ),
        ),
        embedder=dict(
            provider="google", # or openai, ollama, ...
            config=dict(
                model="models/embedding-001",
                task_type="retrieval_document",
                # Optional title for the embeddings can be added here.
                # title="Embeddings",
            ),
        ),
    )
)


## Creating Agents

In [32]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extractin critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    ),
    llm=llm
)

In [33]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    ),
    llm=llm
)

In [34]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    ),
    llm=llm
)

In [35]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    ),
    llm=llm
)

## Creating Tasks

In [36]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

In [37]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

- You can pass a list of tasks as `context` to a task.
- The task then takes into account the output of those tasks in its execution.
- The task will not run until it has the output(s) from those tasks.

In [38]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [39]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


## Creating the Crew

In [40]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [41]:
job_application_inputs = {
    'job_posting_url': 'https://www.karriere.at/jobs/software-engineer',
    'github_url': 'https://github.com/mutabay',
    'personal_writeup': """Mustafa is a software engineer with hands-on experience in various areas of computer science, including web development, IoT, machine learning, DevOps, and natural language processing. Currently, Mustafa is pursuing a Master’s degree in Informatics (AI Specialization) at Alpen-Adria-Universität Klagenfurt with a Technology Scholarship. Mustafa's master thesis research at Infineon Technologies spans the fields of software engineering, artificial intelligence, and distributed systems.

    During the internship at Infineon Technologies, Mustafa contributed to digitalization initiatives in the Quality Management Digitalization team, developing innovative solutions. In Berlin, as an AI Intern at prEUnec GmbH, Mustafa worked on computer vision projects, focusing on deep learning models like YOLO for industrial applications. Mustafa also gained experience as a Full Stack Web Developer in Ankara.

    Mustafa graduated in the top third of my department with honors in Software Engineering from Istanbul Maltepe University and participated in an Erasmus+ exchange program for two semesters in Applied Computer Science at Wroclaw University of Science and Technology. Additionally, in the final year of my bachelor’s studies, Mustafa participated in the Microsoft Imagine program, where Mustafa developed my skills further through innovative projects and collaborative opportunities."""
}

In [ ]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

# Agent: Tech Job Researcher
## Task: Analyze the job posting URL provided (https://www.karriere.at/jobs/software-engineer) to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.
# Agent: Personal Profiler for Engineers
## Task: Compile a detailed personal and professional profile using the GitHub (https://github.com/mutabay) URLs, and personal write-up (Mustafa is a software engineer with hands-on experience in various areas of computer science, including web development, IoT, machine learning, DevOps, and natural language processing. Currently, Mustafa is pursuing a Master’s degree in Informatics (AI Specialization) at Alpen-Adria-Universität Klagenfurt with a Technology Scholarship. Mustafa's master thesis research at Infineon Technologies spans the fields of software engineering, artificial intelligence, and distributed systems.

    During the internship at Infineon Technologies, Mustafa contri

- Dislplay the generated `tailored_resume.md` file.

In [20]:
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

The individual is a Master's student in Informatics (AI Specialization) with practical expertise in software engineering, distributed systems, and artificial intelligence. They have experience in developing end-to-end solutions and are eligible to work in Austria under a post-study visa.

- Dislplay the generated `interview_materials.md` file.

In [21]:
display(Markdown("./interview_materials.md"))

The concept of software engineering involves a structured approach to designing, developing, testing, and maintaining software systems. It requires consideration of various factors such as requirements analysis, design patterns, algorithms, data structures, computer architecture, operating systems, programming languages, database management, software development methodologies, project management, quality assurance, testing, and validation.

In addition to these technical aspects, software engineering also involves understanding the social and organizational context in which software systems are developed and used. This includes knowledge of human-computer interaction, usability, accessibility, and security.

Software engineers use a variety of tools and techniques to develop software systems, including programming languages, integrated development environments (IDEs), version control systems, agile methodologies, and continuous integration and delivery pipelines.

The goal of software engineering is to produce high-quality software systems that meet the needs of users, stakeholders, and other relevant parties. This requires a combination of technical expertise, business acumen, and communication skills.

In conclusion, software engineering is a complex and multifaceted field that involves a range of technical, social, and organizational considerations. It requires a deep understanding of software systems, their development, testing, and maintenance, as well as the ability to work effectively with others in a team environment.